In [1]:
from dataclasses import dataclass
from typing import Any, List, Iterable

# Utils

In [2]:
def get_members(cls):
    return set([att for att in dir(cls) if not att.startswith("_")])

def get_values(cls):
    temp = [getattr(cls, att) for att in dir(cls) if not att.startswith("_")]
    vals = set()
    for item in temp:
        if type(item) in (int,float,str):
            vals.add(item)
        elif type(item) in (set,List,tuple):
            for i in item:
                vals.add(i)
    return vals

def add_meta_data(cls):
    cls.Members = get_members(cls)
    cls.Values  = get_values(cls)
                
def to_tuples(items):
    tuples = set()
    for item in items:
        tpl = tuple(StringUtils.clean(item).split(" "))
        tuples.add(tpl)
    return tuples
    
class Singleton(object):
    __instance = None
    def __new__(cls, *args):
        if cls.__instance is None:
            cls.__instance = object.__new__(cls, *args)
        return cls.__instance

class StringUtils(object):
    
    @staticmethod
    def init_caps(s):
        return s[0].upper() + s[1:]
    
    @staticmethod
    def clean(s):
        if not s:
            return ""
        return str(s).replace("-"," ").strip().lower()

# Entities

In [11]:
class CompassEnum(Singleton):
    North = "north"
    South = "south"
    East  = "east"
    West  = "west"
    
class VerticalEnum(Singleton):
    Above = "above"
    Below = "below"
        
add_meta_data(CompassEnum)
add_meta_data(VerticalEnum)

@dataclass
class Item(object):
    name: str = ""
        
    def __repr__(self):
        return self.name

@dataclass
class Location(object):
    desc: str = "A room"
    name: str = ""
    
    north: Any = None
    south: Any = None
    east:  Any = None
    west:  Any = None

    above: Any = None
    below: Any = None
        
    def add_north(self, locn):
        self.north = locn
        locn.south = self
        
    def add_south(self, locn):
        self.south = locn
        locn.north = self
        
    def add_east(self, locn):
        self.east = locn
        locn.west = self
        
    def add_west(self, locn):
        self.west = locn
        locn.east = self
        
    def __post_init__(self):
        self.desc = self.desc.strip()
        self.name = self.name.strip()
        
        self.items: List[Any]     = []
            
    def get_locations(self):
        locations: List[Any] = []
        if self.north:
            locations.append(CompassEnum.North)
        if self.south:
            locations.append(CompassEnum.South)
        if self.west:
            locations.append(CompassEnum.West)
        if self.east:
            locations.append(CompassEnum.East)
        if self.above:
            locations.append(VerticalEnum.Above)
        if self.below:
            locations.append(VerticalEnum.Below)
        return locations
        
    def describe(self):
        out_val = self.desc
        
        for name in self.get_locations():
            locn = getattr(self,name)
            if name in CompassEnum.Values:
                out_val += f"\n{StringUtils.init_caps(name)} is a {locn.name}. "
            elif name in CompassEnum.Values:
                out_val += f"\nTo the {name} is a {locn.name}. "
            else:
                raise Exception(f"Unknown location name: {name}")

        if len(self.items) > 0:
            if len(self.items) == 1:
                str_items = f"a {self.items[0]}"
            elif len(self.items) > 1:                
                str_items = ", ".join([f"a {i}" for i in self.items[:-1]])
                str_items += f" and a {self.items[-1]}"            
            out_val += f"In the {self.name} you find {str_items}."
        return out_val

# Game State

In [4]:
@dataclass
class Player(Singleton):    
    hp: int         = 100
        
input_list = []
def pop_from_list(s):
    global input_list
    if input_list:
        item = input_list[0]
        input_list = input_list[1:]
        print(item)
        return item
    # print("Input exhausted")
    return "quit"        

class GameState(Singleton):
    def __init__(self):
        self.player = Player()
        self.location = None
        self.inventory = set()
        self.visited = []
        self.parser = Parser()
        
    def process_input(self, user_input):
        # Continue or not?
        return self.parser.parse(user_input)
        
    def process_special_command(self, command):
        if command in SpecialCommandsEnum.Describe:
            self.print_output(self.location.describe())
        elif command in SpecialCommandsEnum.Help:
            # TODO complete this
            self.print_output("Help Requested - TODO")
        elif command == SpecialCommandsEnum.Quit:
            self.print_output("Quiting...")
            return False
        else:
            raise Exception(f"Unrecognized special command: {command}")
        return True
        
    def update_location(self, location):
        self.location = location
        self.visited.append(self.location.name)
        return (True, self.location.describe())
    
    def move_location(self, direction):
        locn = getattr(self.location, direction, None)
        if not locn:
            return (True, f"Cannot move {direction}!")
        return self.update_location(locn)
    
    def print_output(self, s):
        print(s)
        
    def end_game(self):
        self.print_output("Game Over")

class Game(Singleton):
            
    def __init__(self):
        self.play = True
        self.loop_num = 0
        self.gs = GameState()
        
    def run(self, init_location):        
        self.gs.update_location(init_location)
        while self.play and self.loop_num < 1000:            
            self.loop_num += 1
            self.loop()
        self.gs.end_game()
        
    def get_user_input(self, prompt):
        return pop_from_list(prompt)

    def loop(self):
        user_input = self.get_user_input("What do you want to do?")
        self.play = self.gs.process_input(user_input)

# Parser + Enums

In [10]:
RULES = """
north,south,east,west =><CompassDir>
above,up,climb up     =><VerticalUp>
below,down,climb down =><VerticalDown>
sword,torch,key,potion=><PickUpAble>

pick up,pickup        =>get

move|go <CompassDir>  =>[move] <CompassDir>
move|go <VerticalUp>  =>[move] above
<VerticalUp>          =>[move] above
move|go <VerticalDown>=>[move] below
<VerticalDown>        =>[move] below
get <PickUpAble>      =>[pickup] <PickUpAble>

quit,exit             =>[quit]
describe              =>[describe]
help,h,?              =>[help]

""".strip().split("\n")

In [8]:
@dataclass
class Token(object):
    token: str
    data: Any

class Parser(Singleton):
    StopWords = set("a,an,the".split(","))
    
    def __init__(self):
        self.fst = self.build_fst(RULES)
    
    # handles | (or) tokens
    def generate_or_variants(self, tokens):
        l_toks  = [[]]
        for token in tokens:
            new_l = []
            for tok in token.split("|"):            
                for lst in l_toks:
                    new_l.append(lst + [tok])
            l_toks = new_l
        return l_toks

    def build_fst(self, rules):        
        fst = dict()
        for line in rules:
            line = line.strip()
            if not line:
                continue
            
            left, right = line.split("=>")
            left = left.strip()
            right = right.strip()
            
            lhs_phrases = [phrase.strip().split(" ") for phrase in left.split(",")]
            assert "," not in right, "Multiple right hand phrases not supported for now"

            rhs_phrases = [t.strip() for t in right.split(" ") if len(t.strip()) > 0]
            for tokens in lhs_phrases:
                # remove empty tokens
                raw_tokens = [t for t in tokens if len(t.strip()) > 0]
                if not raw_tokens:
                    continue
                
                # Generate permutations of lhs rule when OR chars present
                lst_tokens = self.generate_or_variants(raw_tokens)
                for tokens in lst_tokens:
                    # build dictionary for phrase
                    dct = fst
                    for tok in tokens[:-1]:
                        if tok not in dct:
                            dct[tok] = tuple([None, dict()])
                        dct = dct[tok][1]
                    # last token
                    tok = tokens[-1]
                    if tok not in dct:
                        dct[tok] = tuple([rhs_phrases, dict()])
                    else:
                        rhs_tpl = dct[tok]
                        assert rhs_tpl[0] is None, (left, right, tokens, rhs_tpl)
                        # update tuple
                        dct[tok] = tuple([rhs_phrases, rhs_tpl[1]])
        return fst
    
    def process_rules(self, tokens: List[Token])->List[Token]:
        rule_matched = True
        while rule_matched:
            tokens, rule_matched  = self.process_rules_inner(tokens)
        return tokens
    
    def process_rules_inner(self, tokens: List[Token]):
        
        output = []
        ix = 0
        rule_matched = False
        
        while ix < len(tokens):
            current_tok = tokens[ix]
            if current_tok.token not in self.fst:
                # skip unrecognized for now
                output.append(current_tok)
            else:
                best_rhs = None
                dct = self.fst
                num_tokens_matched = 0 # how long into the tokens array did we go?
                remainder = tokens[ix:] 
                for tok in remainder:
                    
                    if not tok.token in dct:
                        break
                    num_tokens_matched += 1
                    emit, dct = dct[tok.token]
                    if emit:
                        best_rhs = emit
                    # partial match only
                    if not dct:
                        break
                if not best_rhs:
                    output.append(current_tok)
                else:
                    rule_matched = True
                    matched = remainder[:num_tokens_matched]
                    diff = [t.token for t in matched if t.token not in best_rhs]                    
                    str2token = dict([(t.token, t) for t in matched])
                    for str_tok in best_rhs:
                        data = diff
                        if str_tok in str2token:
                            # if token on both sides, carry over the data, e.g. move|go <CompassDir>  =>[move] <CompassDir>
                            data = str2token[str_tok].data                        
                        new_tok = Token(token=str_tok, data=data)
                        output.append(new_tok)
                    ix += num_tokens_matched - 1 # minus 1 as we are about to add one in a sec
            ix += 1
        return output, rule_matched
    
    def parse(self, s):
        gs = GameState()
        tokens = [Token(data=t, token=t) for t in StringUtils.clean(s).split(" ") 
                  if t not in Parser.StopWords]
        
        if not tokens:
            return False
        
        tokens = self.process_rules(tokens)
        print(tokens)
        for i in range(len(tokens)):
            tok = tokens[i]
            if tok.token.startswith("["):
                method = tok.token[1:-1]
                args = [t.data for t in tokens[i+1:]]
                print(f"Parsed: {method}({args[0][0]})")
                return True
        
        gs.print_output(f"Input not recognized: {s}")
        return True
    
    
tokens = "pick up sword"
p = Parser()
valid = p.parse(tokens)
valid

[Token(token='[pickup]', data=['get']), Token(token='<PickUpAble>', data=['sword'])]
Parsed: pickup(sword)


True

# World Generation

In [9]:
def generate_world():
    intro = Location(
        desc="Dazed, you awaken to find yourself in a large, dank cavern. ",
        name="large cavern"
    )
    locn_west = Location(desc="""
You enter a small cave with a low ceiling. Inside there is a dank smell, and a low, rumbling noise coming from one corner of the room.
Wary, you glance over a see a snout poking out from the side of a pile of rocks.
""",
                        name="dragon room")
    
    locn_east = Location(desc="",
                        name="treasure room")
    
    intro.add_west(locn_west)
    intro.add_east(locn_east)
    
    intro.items.append(Item(name="torch"))
    intro.items.append(Item(name="sword"))
    return intro

intro = generate_world()
intro.describe()

NameError: name 'CompassEnum' is not defined

In [ ]:
# input_list = [
#     "move west",
#     "move east",
#     "quit",
# ]

# game = Game()
# game.run(intro)